#### Aula 17

#### Exercício 1

In [4]:
%%writefile random_thrust.cu

#include <thrust/random.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>
#include <vector>

int main() {
  thrust::default_random_engine eng(10);
  thrust::uniform_real_distribution<double> d(20, 30);

  for (int i = 0; i < 10; i++) {
      std::cout << d(eng) << "\n";
  }

}

Overwriting random_thrust.cu


In [5]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o random_thrust random_thrust.cu

In [6]:
!./random_thrust < in1.txt

20.0022
28.5032
20.1353
29.1611
26.7956
28.969
21.4976
29.8008
26.2906
24.3512


#### Exercício 2

In [63]:
%%writefile random_customizado.cu

#include <thrust/random.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>
#include <vector>
using namespace std;

struct rng_gpu {
    __device__ __host__
    double operator() (const int &i) {
        thrust::default_random_engine eng(i*10000000);
        thrust::uniform_real_distribution<double> d(25, 40);
        return d(eng);
    }
};

int main() {
    //DENTRO DO PROGRAMA PRINCIPAL, UMA FORMA DE CHAMAR A SUA FUNCAO
    size_t N = 10;
    thrust::device_vector<double> d_random(N);

    thrust::transform(
        thrust::make_counting_iterator<int>(0),
        thrust::make_counting_iterator<int>(N),
        d_random.begin(),
        rng_gpu()
    );

    for(int i = 0; i < N; i++) {
        cout << d_random[i] << endl;
    }

}

Overwriting random_customizado.cu


In [64]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o random_customizado random_customizado.cu

In [65]:
!./random_customizado

25.0003
36.6904
33.3808
30.0712
26.7616
38.452
35.1424
31.8328
28.5232
25.2136


#### Exercício Monte Carlo GPU

In [87]:
%%writefile pi_montecarlo-GPU.cu

#include <random>
#include <iostream>
#include <vector>
#include <thrust/random.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/iterator/counting_iterator.h>
using namespace std;

struct rng_gpu {
    __device__ __host__
    double operator() (const int &i) {
        thrust::default_random_engine generator(i*10000000);
        thrust::uniform_real_distribution<double> distribution(0.0, 1.0);
        return distribution(generator);
    }
};

int main () {

    long N = 1000000L;
    long sum = 0;
    thrust::host_vector<double> dotsCPU(2*N);
    thrust::device_vector<double> dots(dotsCPU);

    thrust::transform(
        thrust::make_counting_iterator<int>(0),
        thrust::make_counting_iterator<int>(2*N),
        dots.begin(),
        rng_gpu()
    );
  
   for (long i = 0; i < N; i++) {
        double x, y;
      
        x = dots[i];
        y = dots[2*i];
        if (x*x + y*y <= 1) {
            sum++;
        }
    }
    

    double pi = 4.0 * sum / N;
    cout << pi << endl;
    cout << sum << endl;
  
  }

Overwriting pi_montecarlo-GPU.cu


In [88]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o pi_montecarlo-GPU pi_montecarlo-GPU.cu

In [89]:
!./pi_montecarlo-GPU

2.98886
747215
